In [ ]:
import os
from openai import OpenAI

client = OpenAI(base_url="http://localhost:8001/v1", api_key="EMPTY")

models = client.models.list()
model_name = models.data[0].id
print(f"Đang dùng model: {model_name}")

response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system", "content": "Bạn là trợ lý AI hữu ích."},
        {"role": "user", "content": "Giải thích ngắn gọn về thuyết tương đối."},
    ],
    temperature=0.7,
)

print(response.choices[0].message.content)

In [ ]:
from openai import OpenAI

client = OpenAI(base_url="https://models.github.ai/inference", api_key=os.getenv('GITHUB_TOKEN'))

response = client.chat.completions.create(
    model="openai/gpt-4.1",
    messages=[
        {"role": "system", "content": "Bạn là trợ lý AI hữu ích."},
        {"role": "user", "content": "Giải thích ngắn gọn về thuyết tương đối."},
    ],
    temperature=0.7,
)

print(response.choices[0].message.content)

# Function

In [ ]:
from typing import List, Dict, Any, Optional
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage, Document
from pymongo import MongoClient
from langchain_huggingface import HuggingFaceEmbeddings
from duckduckgo_search import DDGS
import torch
import numpy as np
import json
from datetime import datetime

In [ ]:
# ============================================
# CATEGORY MAPPING
# ============================================
CATEGORY_LABELS = {
    0: "Dairy",
    1: "Eggs",
    2: "Fish",
    3: "Gluten",
    4: "Peanuts",
    5: "Sesame",
    6: "Shellfish",
    7: "Soy",
    8: "Tree Nuts",
    9: "Chocolate",
    10: "Cilantro",
    11: "Kale",
    12: "Mayonnaise",
    13: "Mushrooms",
    14: "Mustard",
    15: "Olives",
    16: "Onions",
    17: "Pickles",
    18: "Protein Powder",
    19: "Shakes & Smoothies",
    20: "Sugar",
    21: "Blue Cheese",
    22: "Butter",
    23: "Cheese",
    24: "Cottage Cheese",
    25: "Cream",
    26: "Goat Cheese",
    27: "Milk",
    28: "Whey Powder",
    29: "Yogurt",
    30: "Red Meat",
    31: "Beef",
    32: "Lamb",
    33: "Pork & Bacon",
    34: "Sausages and Luncheon Meats",
    35: "Poultry",
    36: "Chicken",
    37: "Duck",
    38: "Turkey",
    39: "Cod",
    40: "Salmon",
    41: "Sardines",
    42: "Tilapia",
    43: "Trout & Snapper",
    44: "Tuna",
    45: "Clams",
    46: "Crab",
    47: "Lobster",
    48: "Mussels",
    49: "Oysters",
    50: "Scallops",
    51: "Shrimp",
    52: "Squid",
    53: "Vegetables",
    54: "Artichoke",
    55: "Arugula",
    56: "Asparagus",
    57: "Beets",
    58: "Bell Peppers",
    59: "Broccoli",
    60: "Brussel Sprouts",
    61: "Cabbage",
    62: "Carrots",
    63: "Cauliflower",
    64: "Celery",
    65: "Chili Peppers",
    66: "Cucumber",
    67: "Eggplant",
    68: "Garlic",
    69: "Lettuce",
    70: "Potatoes & Yams",
    71: "Radish",
    72: "Spinach",
    73: "Squash",
    74: "Tomato",
    75: "Zucchini",
    76: "Fruit",
    77: "Apple",
    78: "Avocado",
    79: "Banana",
    80: "Blueberries",
    81: "Coconut",
    82: "Dates",
    83: "Grapes",
    84: "Kiwi",
    85: "Lemon",
    86: "Lime",
    87: "Mango",
    88: "Melon",
    89: "Orange",
    90: "Peaches & Plums",
    91: "Pineapple",
    92: "Raisins",
    93: "Raspberries",
    94: "Strawberries",
    95: "Edamame",
    96: "Soy Milk",
    97: "Soy Sauce",
    98: "Tempeh",
    99: "Tofu",
    100: "Grains",
    101: "Barley",
    102: "Bread",
    103: "Breakfast Cereals",
    104: "Corn",
    105: "Oats",
    106: "Pastas",
    107: "Quinoa",
    108: "Rice",
    109: "Rye",
    110: "Wheat",
    111: "Legumes",
    112: "Beans",
    113: "Chickpeas",
    114: "Hummus",
    115: "Lentils",
    116: "Almonds",
    117: "Brazil Nuts",
    118: "Cashews",
    119: "Hazelnuts",
    120: "Pecans",
    121: "Pistachios",
    122: "Walnuts",
    123: "Fish Sauce",
    124: "Honey",
    125: "Ketchup",
    126: "Mayonnaise",
    127: "Mustard",
    128: "Pickles",
    129: "Spices and Herbs",
    130: "Sweets",
    131: "Soups, Sauces, and Gravies",
    132: "Baked Products",
    133: "Beverages",
    134: "Fast Foods",
    135: "Ethnic Foods",
    136: "Supplements",
}


# ============================================
# FOOD RAG PIPELINE
# ============================================
class FoodRAGPipeline:
    """RAG Pipeline cho Food Database"""

    def __init__(
        self, mongodb_uri: str, database_name: str, collection_name: str = "foods"
    ):
        print("🍕 Initializing Food RAG Pipeline...")

        # Embeddings model (shared với User Manual)
        self.embeddings = HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-small",
            model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
            encode_kwargs={"normalize_embeddings": True},
        )

        # MongoDB connection
        self.client = MongoClient(mongodb_uri)
        self.db = self.client[database_name]
        self.collection = self.db[collection_name]

        print(f"   ✓ Connected to {database_name}.{collection_name}")
        print(f"   ✓ Total foods: {self.collection.count_documents({})}")

    def search(self, query: str, k: int = 5, filters: Dict = None) -> List[Document]:
        """
        Vector search trong food database

        Args:
            query: Search query (English hoặc Vietnamese)
            k: Số lượng kết quả
            filters: MongoDB filters (e.g., {"property.isBreakfast": True})

        Returns:
            List of LangChain Documents
        """
        # Embed query
        query_embedding = self.embeddings.embed_query(query)

        # Build pipeline
        pipeline = [
            {
                "$vectorSearch": {
                    "index": "vector_index",
                    "path": "embedding",
                    "queryVector": query_embedding,
                    "numCandidates": k * 20,
                    "limit": k,
                }
            }
        ]

        # Add filters if provided
        if filters:
            pipeline.append({"$match": filters})

        # Project fields
        pipeline.append(
            {
                "$project": {
                    "name": 1,
                    "text_content": 1,
                    "categories": 1,
                    "nutrition": 1,
                    "property": 1,
                    "score": {"$meta": "vectorSearchScore"},
                }
            }
        )

        # Execute search
        results = list(self.collection.aggregate(pipeline))

        # Convert to LangChain Documents
        documents = []
        for result in results:
            # Format nutrition info
            nutrition = result.get("nutrition", {})
            nutrition_text = f"""
Calories: {nutrition.get('calories', 'N/A')} kcal
Protein: {nutrition.get('proteins', 'N/A')}g
Carbs: {nutrition.get('carbs', 'N/A')}g
Fat: {nutrition.get('fats', 'N/A')}g
Fiber: {nutrition.get('fiber', 'N/A')}g
"""

            # Format properties
            properties = result.get("property", {})
            meal_types = []
            if properties.get("isBreakfast"):
                meal_types.append("Breakfast")
            if properties.get("isLunch"):
                meal_types.append("Lunch")
            if properties.get("isDinner"):
                meal_types.append("Dinner")
            if properties.get("isSnack"):
                meal_types.append("Snack")

            # Format categories
            categories = result.get("categories", [])
            category_names = [
                CATEGORY_LABELS.get(c, f"Category {c}") for c in categories
            ]

            # Create document content
            content = f"""
Food Name: {result.get('name', 'Unknown')}

Description:
{result.get('text_content', 'No description available')}

Nutritional Information:
{nutrition_text}

Meal Types: {', '.join(meal_types) if meal_types else 'N/A'}
Categories: {', '.join(category_names)}
Cooking Time: {properties.get('totalTime', 'N/A')} minutes
Complexity: {properties.get('complexity', 'N/A')}
"""

            # Create Document
            doc = Document(
                page_content=content.strip(),
                metadata={
                    "name": result.get("name"),
                    "score": result.get("score", 0),
                    "categories": category_names,
                    "nutrition": nutrition,
                    "properties": properties,
                    "source": "food_database",
                },
            )
            documents.append(doc)

        return documents

    def close(self):
        """Close MongoDB connection"""
        self.client.close()


# ============================================
# USER MANUAL RAG PIPELINE
# ============================================
class UserManualRAGPipeline:
    """RAG Pipeline cho User Manual - sử dụng DocumentEmbeddingPipeline hiện có"""

    def __init__(
        self,
        mongodb_uri: str,
        database_name: str,
        collection_name: str = "user_manual_embeddings",
    ):
        print("📚 Initializing User Manual RAG Pipeline...")

        # Embeddings model (shared)
        self.embeddings = HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-small",
            model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
            encode_kwargs={"normalize_embeddings": True},
        )

        # MongoDB connection
        self.client = MongoClient(mongodb_uri)
        self.db = self.client[database_name]
        self.collection = self.db[collection_name]

        print(f"   ✓ Connected to {database_name}.{collection_name}")
        print(f"   ✓ Total chunks: {self.collection.count_documents({})}")

    def semantic_search(
        self, query: str, top_k: int = 3, filters: Dict = None
    ) -> List[Document]:
        """
        Semantic search trong user manual

        Args:
            query: Search query
            top_k: Number of results
            filters: MongoDB filters (e.g., {"metadata.section_type": "feature"})

        Returns:
            List of LangChain Documents
        """
        # Embed query
        query_vector = self.embeddings.embed_query(query)

        try:
            # Build pipeline
            pipeline = [
                {
                    "$vectorSearch": {
                        "index": "vector_index",
                        "path": "embedding",
                        "queryVector": query_vector,
                        "numCandidates": top_k * 10,
                        "limit": top_k,
                    }
                }
            ]

            # Add filters
            if filters:
                pipeline.append({"$match": filters})

            # Project results
            pipeline.append(
                {
                    "$project": {
                        "text": 1,
                        "metadata": 1,
                        "score": {"$meta": "vectorSearchScore"},
                    }
                }
            )

            results = list(self.collection.aggregate(pipeline))

        except Exception as e:
            print(f"⚠️  Atlas Vector Search failed: {e}")
            print("   Falling back to manual search...")

            # Fallback: Manual search
            query_dict = filters if filters else {}
            all_docs = list(self.collection.find(query_dict))

            similarities = []
            for doc in all_docs:
                similarity = np.dot(query_vector, doc["embedding"])
                similarities.append(
                    {
                        "text": doc["text"],
                        "metadata": doc["metadata"],
                        "score": float(similarity),
                    }
                )

            similarities.sort(key=lambda x: x["score"], reverse=True)
            results = similarities[:top_k]

        # Convert to LangChain Documents
        documents = []
        for r in results:
            doc = Document(
                page_content=r["text"],
                metadata={
                    **r["metadata"],
                    "score": r["score"],
                    "source": "user_manual",
                },
            )
            documents.append(doc)

        return documents

    def close(self):
        """Close MongoDB connection"""
        self.client.close()


# ============================================
# LANGCHAIN AGENT
# ============================================
class MealPlannerAgent:
    """LangChain Agent tích hợp Food RAG + User Manual RAG + Web Search"""

    def __init__(
        self,
        food_rag: FoodRAGPipeline,
        manual_rag: UserManualRAGPipeline,
        vllm_base_url: str,
        vllm_model_name: str,
        vllm_api_key: str = "EMPTY",
    ):
        """
        Initialize Agent

        Args:
            food_rag: FoodRAGPipeline instance
            manual_rag: UserManualRAGPipeline instance
            vllm_base_url: VLLM server URL
            vllm_model_name: Model name in VLLM
            vllm_api_key: API key (default "EMPTY")
        """
        print("🤖 Initializing Meal Planner Agent...")

        self.food_rag = food_rag
        self.manual_rag = manual_rag

        # Initialize LLM
        self.llm = ChatOpenAI(
            base_url=vllm_base_url,
            api_key=vllm_api_key,
            model=vllm_model_name,
            temperature=0.7,
        )

        # Create tools
        self.tools = self._create_tools()

        # Create agent
        self.agent = self._create_agent()

        print("   ✓ Agent initialized with all tools")

    def _create_tools(self) -> List[Tool]:
        """Create LangChain tools"""

        return [
            Tool(
                name="search_food_database",
                func=self._search_foods,
                description="""
                Search the food database for recipes, ingredients, and nutritional information.
                Use this when users ask about:
                - Finding recipes or meals
                - Nutritional information
                - Specific ingredients or food items
                - Meal suggestions for breakfast/lunch/dinner
                - Dietary requirements (high protein, low carb, etc.)

                Input: Natural language query about food
                Example: "high protein breakfast recipes" or "món ăn sáng giàu đạm"
                """,
            ),
            Tool(
                name="search_user_manual",
                func=self._search_manual,
                description="""
                Search the user manual for app features and how-to guides.
                Use this when users ask about:
                - How to use app features
                - Navigation and UI questions
                - Account management
                - Settings and preferences
                - Troubleshooting

                Input: Question about app functionality
                Example: "how to create meal plan" or "làm sao tạo thực đơn"
                """,
            ),
            Tool(
                name="web_search",
                func=self._web_search,
                description="""
                Search the web for current information, trends, and general knowledge.
                Use this when users ask about:
                - Recent health/nutrition trends
                - Latest dietary guidelines
                - Current food/restaurant recommendations
                - News about nutrition or cooking

                Input: Search query for web
                Example: "nutrition trends 2024" or "xu hướng dinh dưỡng 2024"
                """,
            ),
            Tool(
                name="generate_frontend_command",
                func=self._generate_command,
                description="""
                Generate commands to execute actions on the frontend.
                Use this when users want to:
                - Create/edit meal plans
                - Add items to grocery list
                - Save recipes to favorites
                - Navigate to specific pages
                - View detailed information

                Input: JSON string with action and parameters
                Example: '{"action": "create_meal_plan", "params": {"date": "2024-01-20"}}'

                Available actions:
                - create_meal_plan: Create new meal plan
                - add_to_grocery_list: Add items to shopping list
                - save_recipe: Save recipe to favorites
                - navigate_to: Navigate to page
                - show_nutrition_info: Display nutrition details
                - filter_recipes: Apply recipe filters
                """,
            ),
        ]

    def _search_foods(self, query: str) -> str:
        """Search food database"""
        try:
            # Extract filters from query if needed
            filters = self._extract_food_filters(query)

            # Search
            results = self.food_rag.search(query, k=5, filters=filters)

            if not results:
                return "Không tìm thấy món ăn nào phù hợp. / No foods found matching your query."

            # Format results
            formatted = []
            for i, doc in enumerate(results, 1):
                metadata = doc.metadata
                formatted.append(
                    f"""
{i}. {metadata.get('name', 'Unknown')}
   Score: {metadata.get('score', 0):.3f}
   Categories: {', '.join(metadata.get('categories', [])[:3])}
   Calories: {metadata.get('nutrition', {}).get('calories', 'N/A')} kcal
   Protein: {metadata.get('nutrition', {}).get('proteins', 'N/A')}g

   {doc.page_content[:200]}...
"""
                )

            return "\n".join(formatted)

        except Exception as e:
            return f"Error searching food database: {str(e)}"

    def _extract_food_filters(self, query: str) -> Optional[Dict]:
        """Extract MongoDB filters from natural language query"""
        query_lower = query.lower()
        filters = {}

        # Meal type filters
        if any(word in query_lower for word in ["breakfast", "sáng", "bữa sáng"]):
            filters["property.isBreakfast"] = True
        elif any(word in query_lower for word in ["lunch", "trưa", "bữa trưa"]):
            filters["property.isLunch"] = True
        elif any(word in query_lower for word in ["dinner", "tối", "bữa tối"]):
            filters["property.isDinner"] = True
        elif any(word in query_lower for word in ["snack", "ăn vặt"]):
            filters["property.isSnack"] = True

        # Dietary filters
        if any(
            word in query_lower for word in ["high protein", "giàu protein", "giàu đạm"]
        ):
            filters["property.isHighProtein"] = True
        elif any(
            word in query_lower for word in ["low carb", "ít carb", "ít tinh bột"]
        ):
            filters["property.isLowCarb"] = True
        elif any(word in query_lower for word in ["low fat", "ít béo", "ít chất béo"]):
            filters["property.isLowFat"] = True
        elif any(word in query_lower for word in ["high fiber", "nhiều chất xơ"]):
            filters["property.isHighFiber"] = True

        return filters if filters else None

    def _search_manual(self, query: str) -> str:
        """Search user manual"""
        try:
            results = self.manual_rag.semantic_search(query, top_k=3)

            if not results:
                return "Không tìm thấy thông tin trong hướng dẫn sử dụng. / No relevant documentation found."

            # Format results
            formatted = []
            for i, doc in enumerate(results, 1):
                metadata = doc.metadata
                formatted.append(
                    f"""
{i}. [Score: {metadata.get('score', 0):.3f}]
   Section: {metadata.get('section_title', 'N/A')}
   Path: {metadata.get('full_path', 'N/A')}

   {doc.page_content[:300]}...
"""
                )

            return "\n".join(formatted)

        except Exception as e:
            return f"Error searching manual: {str(e)}"

    def _web_search(self, query: str) -> str:
        """Search the web"""
        try:
            with DDGS() as ddgs:
                results = list(ddgs.text(query, max_results=5))

            if not results:
                return "Không tìm thấy kết quả trên web. / No web results found."

            # Format results
            formatted = []
            for i, result in enumerate(results, 1):
                formatted.append(
                    f"""
{i}. {result['title']}
   {result['body'][:200]}...
   URL: {result['href']}
"""
                )

            return "\n".join(formatted)

        except Exception as e:
            return f"Error performing web search: {str(e)}"

    def _generate_command(self, command_spec: str) -> str:
        """Generate frontend command"""
        try:
            spec = json.loads(command_spec)

            command = {
                "type": "FRONTEND_COMMAND",
                "action": spec.get("action"),
                "params": spec.get("params", {}),
                "timestamp": datetime.now().isoformat(),
            }

            return json.dumps(command, ensure_ascii=False, indent=2)

        except Exception as e:
            return f"Error generating command: {str(e)}"

    def _create_agent(self) -> AgentExecutor:
        """Create LangChain agent"""

        system_prompt = """Bạn là trợ lý AI thông minh cho ứng dụng lập kế hoạch bữa ăn NutriPlan.

Bạn có quyền truy cập vào:
1. **Cơ sở dữ liệu thực phẩm** - Công thức nấu ăn, thông tin dinh dưỡng
2. **Hướng dẫn sử dụng** - Tính năng và cách sử dụng ứng dụng
3. **Tìm kiếm web** - Thông tin mới nhất về dinh dưỡng
4. **Tạo lệnh frontend** - Thực hiện các tác vụ trong ứng dụng

Nhiệm vụ của bạn:
- Trả lời câu hỏi về thực phẩm, dinh dưỡng bằng cơ sở dữ liệu
- Hướng dẫn sử dụng ứng dụng bằng tài liệu
- Tìm kiếm thông tin mới trên web khi cần
- Tạo lệnh frontend khi người dùng muốn thực hiện hành động

Nguyên tắc:
- Luôn thân thiện, hữu ích và chính xác
- Ưu tiên nguồn dữ liệu nội bộ trước khi tìm kiếm web
- Khi tạo lệnh frontend, giải thích rõ ràng hành động sẽ thực hiện
- Trả lời bằng tiếng Việt nếu người dùng hỏi tiếng Việt
- Trích dẫn nguồn khi sử dụng thông tin từ web

Lệnh frontend có sẵn:
- create_meal_plan: Tạo kế hoạch bữa ăn mới
- add_to_grocery_list: Thêm vào danh sách mua sắm
- save_recipe: Lưu công thức yêu thích
- navigate_to: Điều hướng đến trang
- show_nutrition_info: Hiển thị thông tin dinh dưỡng
- filter_recipes: Lọc công thức nấu ăn
"""

        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                MessagesPlaceholder(variable_name="chat_history", optional=True),
                ("human", "{input}"),
                MessagesPlaceholder(variable_name="agent_scratchpad"),
            ]
        )

        agent = create_openai_tools_agent(self.llm, self.tools, prompt)

        return AgentExecutor(
            agent=agent,
            tools=self.tools,
            verbose=True,
            max_iterations=5,
            handle_parsing_errors=True,
        )

    def chat(self, message: str, chat_history: List[Dict] = None) -> Dict[str, Any]:
        """
        Process user message

        Args:
            message: User's message
            chat_history: Previous conversation

        Returns:
            Dict with 'response' and 'commands'
        """
        # Convert chat history
        history = []
        if chat_history:
            for msg in chat_history:
                if msg["role"] == "user":
                    history.append(HumanMessage(content=msg["content"]))
                else:
                    history.append(AIMessage(content=msg["content"]))

        # Execute agent
        result = self.agent.invoke({"input": message, "chat_history": history})

        # Extract commands
        response_text = result["output"]
        commands = self._extract_commands(response_text)

        return {"response": response_text, "commands": commands}

    def _extract_commands(self, response: str) -> List[Dict]:
        """Extract frontend commands from response"""
        import re

        commands = []
        json_pattern = r'\{[^{}]*"type"\s*:\s*"FRONTEND_COMMAND"[^{}]*\}'
        matches = re.finditer(json_pattern, response)

        for match in matches:
            try:
                command = json.loads(match.group())
                commands.append(command)
            except:
                continue

        return commands


# ============================================
# MAIN PIPELINE CLASS
# ============================================
class CompleteMealPlannerPipeline:
    """Complete pipeline tích hợp tất cả components"""

    def __init__(
        self,
        mongodb_uri: str,
        database_name: str,
        food_collection: str = "foods",
        manual_collection: str = "user_manual_embeddings",
        vllm_base_url: str = "http://localhost:8001/v1",
        vllm_model_name: str = "your-model-name",
        vllm_api_key: str = "EMPTY",
    ):
        """
        Initialize complete pipeline

        Args:
            mongodb_uri: MongoDB connection string
            database_name: Database name
            food_collection: Food collection name
            manual_collection: Manual collection name
            vllm_base_url: VLLM server URL
            vllm_model_name: Model name
            vllm_api_key: API key
        """
        print("=" * 70)
        print("🚀 INITIALIZING COMPLETE MEAL PLANNER PIPELINE")
        print("=" * 70)

        # Initialize RAG pipelines
        self.food_rag = FoodRAGPipeline(
            mongodb_uri=mongodb_uri,
            database_name=database_name,
            collection_name=food_collection,
        )

        self.manual_rag = UserManualRAGPipeline(
            mongodb_uri=mongodb_uri,
            database_name=database_name,
            collection_name=manual_collection,
        )

        # Initialize agent
        self.agent = MealPlannerAgent(
            food_rag=self.food_rag,
            manual_rag=self.manual_rag,
            vllm_base_url=vllm_base_url,
            vllm_model_name=vllm_model_name,
            vllm_api_key=vllm_api_key,
        )

        print("=" * 70)
        print("✅ PIPELINE READY")
        print("=" * 70)

    def chat(self, message: str, chat_history: List[Dict] = None) -> Dict[str, Any]:
        """Main chat interface"""
        return self.agent.chat(message, chat_history)

    def search_food(self, query: str, k: int = 5, filters: Dict = None):
        """Direct food search"""
        return self.food_rag.search(query, k, filters)

    def search_manual(self, query: str, top_k: int = 3, filters: Dict = None):
        """Direct manual search"""
        return self.manual_rag.semantic_search(query, top_k, filters)

    def close(self):
        """Close all connections"""
        self.food_rag.close()
        self.manual_rag.close()
        print("✓ All connections closed")

# Langchain

In [ ]:
class MealPlannerAgent:
    def __init__(
        self,
        food_rag_pipeline,
        manual_rag_pipeline,
        base_url: str,
        model_name: str,
        api_key: str = "EMPTY",
    ):
        """
        Initialize the agent with RAG pipelines and VLLM connection info

        Args:
            food_rag_pipeline: Your MongoDB vector search for foods
            manual_rag_pipeline: Your semantic search for user manual
            base_url: VLLM server URL (e.g., "http://localhost:8001/v1")
            model_name: Model name in VLLM
            api_key: API key (default "EMPTY" for local VLLM)
        """
        self.food_rag = food_rag_pipeline
        self.manual_rag = manual_rag_pipeline

        # Initialize LLM pointing to your VLLM server
        self.llm = ChatOpenAI(
            base_url=base_url, api_key=api_key, model=model_name, temperature=0.7
        )

        # Create tools
        self.tools = self._create_tools()

        # Create agent
        self.agent = self._create_agent()

    def _create_tools(self) -> List[Tool]:
        """Create LangChain tools for the agent"""

        tools = [
            Tool(
                name="search_food_database",
                func=self._search_foods,
                description="""
                Search the food database for recipes, ingredients, nutritional info.
                Input should be a search query about food, recipes, or nutrition.
                Example: "high protein breakfast recipes" or "vegetarian meals"
                Returns relevant food items with details.
                """,
            ),
            Tool(
                name="search_user_manual",
                func=self._search_manual,
                description="""
                Search the user manual for app features and how-to guides.
                Input should be a question about app functionality.
                Example: "how to create meal plan" or "guest features"
                Returns relevant documentation sections.
                """,
            ),
            Tool(
                name="web_search",
                func=self._web_search,
                description="""
                Search the web for current information not in the database.
                Input should be a search query for recent/general information.
                Example: "latest nutrition guidelines 2024" or "trending diets"
                Returns web search results.
                """,
            ),
            Tool(
                name="generate_frontend_command",
                func=self._generate_command,
                description="""
                Generate a command to execute actions on the frontend.
                Input should be a JSON string describing the desired action.
                Example: '{"action": "create_meal_plan", "params": {"date": "2024-01-20"}}'
                Returns a formatted command object for the frontend.
                """,
            ),
        ]

        return tools

    def _search_foods(self, query: str) -> str:
        """Search food database using MongoDB vector search"""
        try:
            results = self.food_rag(query, k=5)  # Your mongodb_vector_search

            if not results:
                return "No food items found for this query."

            formatted_results = []
            for i, doc in enumerate(results, 1):
                content = f"{i}. {doc.page_content[:200]}..."
                if hasattr(doc, "metadata"):
                    content += f"\n   Metadata: {doc.metadata}"
                formatted_results.append(content)

            return "\n\n".join(formatted_results)
        except Exception as e:
            return f"Error searching food database: {str(e)}"

    def _search_manual(self, query: str) -> str:
        """Search user manual using semantic search"""
        try:
            results = self.manual_rag.semantic_search(query, top_k=3)

            if not results:
                return "No relevant documentation found."

            formatted_results = []
            for i, result in enumerate(results, 1):
                content = f"{i}. [Score: {result['score']:.2f}]\n"
                content += f"   Path: {result['metadata'].get('full_path', 'N/A')}\n"
                content += f"   Content: {result['text'][:300]}..."
                formatted_results.append(content)

            return "\n\n".join(formatted_results)
        except Exception as e:
            return f"Error searching manual: {str(e)}"

    def _web_search(self, query: str) -> str:
        """Search the web using DuckDuckGo"""
        try:
            with DDGS() as ddgs:
                results = list(ddgs.text(query, max_results=5))

            if not results:
                return "No web results found."

            formatted_results = []
            for i, result in enumerate(results, 1):
                content = f"{i}. {result['title']}\n"
                content += f"   {result['body'][:200]}...\n"
                content += f"   URL: {result['href']}"
                formatted_results.append(content)

            return "\n\n".join(formatted_results)
        except Exception as e:
            return f"Error performing web search: {str(e)}"

    def _generate_command(self, command_spec: str) -> str:
        """Generate frontend command from specification"""
        try:
            # Parse the command specification
            spec = json.loads(command_spec)

            # Create standardized command format
            command = {
                "type": "FRONTEND_COMMAND",
                "action": spec.get("action"),
                "params": spec.get("params", {}),
                "timestamp": None,  # Will be set by frontend
            }

            return json.dumps(command, ensure_ascii=False, indent=2)
        except Exception as e:
            return f"Error generating command: {str(e)}"

    def _create_agent(self) -> AgentExecutor:
        """Create the LangChain agent with tools"""

        # System prompt for the agent
        system_prompt = """You are an intelligent meal planning assistant with access to:
1. A food database with recipes and nutritional information
2. User manual documentation for the meal planning app
3. Web search for current information
4. Ability to generate frontend commands

Your responsibilities:
- Answer questions about food, recipes, and nutrition using the food database
- Help users understand app features using the user manual
- Search the web for information not in your databases
- Generate frontend commands when users want to perform actions

When generating frontend commands, use this format:
{{"action": "action_name", "params": {{"key": "value"}}}}

Available frontend actions:
- create_meal_plan: Create a new meal plan
- add_to_grocery_list: Add items to grocery list
- save_recipe: Save a recipe to favorites
- navigate_to: Navigate to a specific page
- show_nutrition_info: Display nutritional information

Always be helpful, accurate, and provide sources when using external information.
If you need to perform an action on the frontend, use the generate_frontend_command tool.
"""

        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                MessagesPlaceholder(variable_name="chat_history", optional=True),
                ("human", "{input}"),
                MessagesPlaceholder(variable_name="agent_scratchpad"),
            ]
        )

        agent = create_openai_tools_agent(self.llm, self.tools, prompt)

        return AgentExecutor(
            agent=agent,
            tools=self.tools,
            verbose=True,
            max_iterations=5,
            handle_parsing_errors=True,
        )

    def chat(self, message: str, chat_history: List[Dict] = None) -> Dict[str, Any]:
        """
        Process user message and return response with optional frontend commands

        Args:
            message: User's input message
            chat_history: Previous conversation history

        Returns:
            Dict with 'response' and optional 'commands' for frontend
        """
        # Convert chat history to LangChain format
        history = []
        if chat_history:
            for msg in chat_history:
                if msg["role"] == "user":
                    history.append(HumanMessage(content=msg["content"]))
                else:
                    history.append(AIMessage(content=msg["content"]))

        # Execute agent
        result = self.agent.invoke({"input": message, "chat_history": history})

        # Parse response for frontend commands
        response_text = result["output"]
        commands = self._extract_commands(response_text)

        return {"response": response_text, "commands": commands}

    def _extract_commands(self, response: str) -> List[Dict]:
        """Extract frontend commands from agent response"""
        commands = []

        # Look for JSON command blocks in response
        import re

        json_pattern = r'\{[^{}]*"type"\s*:\s*"FRONTEND_COMMAND"[^{}]*\}'
        matches = re.finditer(json_pattern, response)

        for match in matches:
            try:
                command = json.loads(match.group())
                commands.append(command)
            except:
                continue

        return commands


# ============================================
# USAGE EXAMPLE
# ============================================

if __name__ == "__main__":
    # Initialize pipeline
    pipeline = CompleteMealPlannerPipeline(
        mongodb_uri="mongodb+srv://your-connection-string",
        database_name="test",
        food_collection="foods",
        manual_collection="llm_documents",
        vllm_base_url="http://localhost:8001/v1",
        vllm_model_name="Qwen/Qwen3-0.6B",
        vllm_api_key="EMPTY",
    )

    # Chat history storage
    chat_history = []

    print("\n🤖 NutriPlan AI Assistant is ready!")
    print("Type 'exit' to quit.")
    print("-" * 60)

    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            break

        # Call pipeline
        result = pipeline.chat(user_input, chat_history)

        # Extract response
        response_text = result["response"]
        commands = result["commands"]

        print("\nAssistant:")
        print(response_text)

        # If frontend commands exist
        if commands:
            print("\n📦 Frontend Commands Detected:")
            for cmd in commands:
                print(json.dumps(cmd, ensure_ascii=False, indent=2))

        # Save history
        chat_history.append({"role": "user", "content": user_input})
        chat_history.append({"role": "assistant", "content": response_text})

    # Cleanup
    pipeline.close()
    print("👋 Goodbye!")